In [1]:
import re
import glob
import pickle
import pandas as pd

In [2]:
posts = glob.glob('data/*.p')

In [3]:
print(posts[0])

data/lexokan@live.unc.edu_posts_j5wwaj87hvu6af.p


In [4]:
# for fp in posts:
#     df = pickle.load(open(fp, "rb"))
#     pd.DataFrame(df)
# df = pd.DataFrame(df)
# print(df.iloc[2])

In [5]:
drop_cols = ['nr' , 'data', 'bucket_order', 'tag_good_arr', 'request_instructor', 'request_instructor_me', 'bookmarked', 'my_favorite', 'is_bookmarked', 
            'is_tag_good', 'q_edits', 'i_edits', 's_edits', 't', 'no_answer_followup', 'change_log', 'default_anonymity']

df = (pd.DataFrame(pickle.load(open(posts[0], "rb")))
        .drop(columns=drop_cols)
     )

In [6]:
df['subject'] = df['history'].apply(lambda x: x[0]['subject'])
df['is_student'] = df['tags'].apply(lambda x: 'student' in x)
df['is_instructor'] = df['tags'].apply(lambda x: 'instructor-note' in x)
df['created'] = pd.to_datetime(df['created'])

# possibly add `http\S+` to the regex
# Remove HTML from text column
df['text'] = df['history'].apply(lambda x: re.sub('<[^<]+?>|\n', ' ', x[0]['content']))

In [7]:
# Confirmed that a post is either tagged as `instructor` or `student` but not both.
is_instructor_counts = df['is_instructor'].value_counts()
is_student_counts = df['is_student'].value_counts()

assert is_student_counts[0] == is_instructor_counts[1]
assert is_student_counts[1] == is_instructor_counts[0]

In [8]:
def num_children(d: dict):
    if 'children' in d:
        yield d['created']
    for k in d:
        if isinstance(d[k], list) and k == 'children':
            for i in d[k]:
                for j in num_children(i):
                    yield j
                    
df['num_children'] = df['children'].apply(lambda x: len(list(num_children(x[0]))) if len(x) > 0 else 0)

In [9]:
df

,folders,created,bucket_name,history,type,tags,tag_good,unique_views,children,id,config,status,upvote_ids,num_favorites,no_answer,subject,is_student,is_instructor,text,num_children
0,[general],2017-12-05 19:59:43+00:00,Pinned,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, pin]",[],244,"[{'anon': 'stud', 'folders': [], 'data': {'emb...",jau1tihlbb82ou,{'is_announcement': 1},active,[],1,NaN,Last Call For Style Regrade Requests,False,True,Any code style regrade requests should be in ...,4
1,[exams],2017-12-05 12:27:44+00:00,Pinned,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[exams, instructor-note, pin]",[],268,[],jatlo9mj8eh16r,{'is_announcement': 1},active,[],0,NaN,Final Exam Logistics,False,True,The final exam for our class will be at 8 AM ...,0
2,[general],2017-11-16 19:23:45+00:00,Pinned,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, pin]",[],258,[],ja2v62y52r47iw,{'is_announcement': 1},active,[],0,NaN,Material for recitation tomorrow,False,True,Below are links to the materials you&#39;ll n...,0
3,[general],2017-11-16 14:02:23+00:00,Pinned,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, pin]",[],257,[],ja2josndfk6z7,{'is_announcement': 1},active,[],2,NaN,Grading issue punchlist form,False,True,If you have any sort of grade / course issue ...,0
4,[assignment7],2017-11-11 03:20:29+00:00,Pinned,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[assignment7, instructor-note, pin]","[{'role': 'student', 'name': 'Justin Do', 'adm...",260,"[{'anon': 'stud', 'folders': [], 'data': {'emb...",j9urk22tlgx5jy,{'is_announcement': 1},active,[],0,NaN,Autograder machine having problems,False,True,Something seems to be wrong with the machine....,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,[general],2017-08-22 02:30:32+00:00,Today,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note]",[],302,[],j6mz3tjz9jy2cx,{},active,[],11,NaN,Pre-Lecture Readings / Videos / Exercises,False,True,(8/24) Lecture 2 http://www.brighthub.c...,0
851,[general],2017-08-22 00:55:39+00:00,Today,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, past-class, private]",[],286,[],j6mvpt4f9yu3a4,{},active,[],3,NaN,"Tutorials, Articles, Other On-line Resources",False,True,Basic tutorials on getting started with Java ...,0
852,[general],2017-08-22 00:54:35+00:00,Today,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[general, instructor-note, past-class, private]",[],281,[],j6mvofglgfc2v2,{},active,[],3,NaN,Tools and Reference Documents,False,True,Java SDK http://www.oracle.com/technetwork...,0
853,"[assignment1, assignment2, assignment3, assign...",2017-08-16 16:11:20+00:00,Today,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",note,"[assignment1, assignment2, assignment3, assign...",[],320,[],j6f7s9qfj841y3,{},active,[],26,NaN,Assignment Schedule,False,True,All assignments are due at 11:59:59 PM ...,0


In [10]:
df['history'][0]

[{'anon': 'no',
  'uid': 'hbpawagy1su7ok',
  'subject': 'Last Call For Style Regrade Requests',
  'created': '2017-12-05T19:59:43Z',
  'content': '<p>Any code style regrade requests should be in by tomorrow (Wednesday, 12/6) night. I will turn off regrade requests sometime after midnight and then the TAs should have them processed in a day or so.</p>\n<p></p>\n<p>#pin</p>'}]

In [11]:
# Get post text in nique columnbs
# diff values of config?
# Check status = active
# transform to get length of upvote_ids

# Question or Answer? 
# Num votes/good question
# where does anon equal yes

Data:
    - Timestamp
    - Userid (student or instructor, throw out announcements)
    - Question or answer? 
    - Votes (good question)
    - Question Text

In [12]:
df['upvote_ids'].count()

854